<a href="https://colab.research.google.com/github/gpandu/GAN/blob/main/GPT_from_scratch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Tokenization:



*   Subword Tokenization : Keep frequent words and break rearer words into subwords
*   A statastical Alogrothm learns how to do this based on corpus.

> Ex: Listeria ---> "list" , "eria"

> "list" is common word likely to be found in dictionary


*   Tokenization has better chance of handling OOV words while decreasing the size of the overall dictionary.   







